In [2]:
# loading a dataset
import pandas as pd
df = pd.read_csv('datasets/nyu_data/selected/selected_augmented.csv')

In [3]:
# check if gpu is available to pytorch
import torch
torch.cuda.is_available()

True

In [4]:
from depth_anything.dpt import DepthAnything
from depth_anything.util.transform import Resize, NormalizeImage, PrepareForNet
from torchvision.transforms import Compose

In [5]:
model = DepthAnything.from_pretrained("LiheYoung/depth_anything_vitl14")

xFormers not available
xFormers not available


In [6]:
transform = Compose([
        Resize(
            width=518,
            height=518,
            resize_target=False,
            keep_aspect_ratio=True,
            ensure_multiple_of=14,
            resize_method='lower_bound',
            image_interpolation_method=cv2.INTER_CUBIC,
        ),
        NormalizeImage(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        PrepareForNet(),
    ])

In [7]:
import numpy as np
from PIL import Image
import cv2
import torch

In [8]:
# writing a function to calculate the mean squared error of an image given its path
def mse(image_path,ground_truth_path):
    image = Image.open(image_path)
    image = np.array(image)
    image = (image - image.min()) / (image.max() - image.min())
    image = transform({'image': image})['image']
    image = torch.from_numpy(image).unsqueeze(0)
    depth = model(image)
    depth = depth.squeeze(0).detach().cpu().numpy()
    depth = (depth - depth.min()) / (depth.max() - depth.min())
    depth = (depth * 255).astype(np.uint8)
    depth = Image.fromarray(depth)
    depth = np.array(depth)
    depth = 255 - depth
    depth = (depth - depth.min())/(depth.max() - depth.min())
    ground_truth = Image.open(ground_truth_path)
    ground_truth = np.array(ground_truth)
    ground_truth = Image.fromarray(ground_truth).resize((686, 518))
    ground_truth = np.array(ground_truth)
    ground_truth  = (ground_truth - ground_truth.min())/(ground_truth.max() - ground_truth.min())
    mse = np.mean((depth - ground_truth) ** 2)
    return mse

In [11]:
# randomly selecting 5000 images and putting them in a dataset
import random
df_new = df.sample(n=5000)
df_new.to_csv('datasets/nyu_data/selected/selected_5000.csv', index=False)

In [12]:
# writing a for loop to iterate over all the images and calculate the mean squared error for each image and appending it to a list
from tqdm import tqdm
mse_list = []
for i in tqdm(range(df_new.shape[0])):
    mse_list.append(mse("datasets/nyu_data/selected/"+df_new.iloc[i,0], "datasets/nyu_data/selected/"+df_new.iloc[i,1]))

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5000/5000 [8:48:12<00:00,  6.34s/it]


In [13]:
# appending the mse_list to the dataframe
df_new['mse'] = mse_list

In [14]:
df_new.head()

,images,depths,mse
1924,colours/172.jpg,depth/172.png,0.049538
34732,colours_blurred/5_5/49.jpg,depth/49.png,0.068819
15538,colours_noisy/0_08/8.jpg,depth/8.png,0.049201
17967,colours_noisy/0_08/00810_colors.png,depth/00810_depth.png,0.111913
4374,colours_noisy/0_01/485.jpg,depth/485.png,0.161699


In [15]:
# saving the dataframe
df_new.to_csv('datasets/nyu_data/selected/selected_5000_mse.csv', index=False)

In [ ]:
# finding the mean mse of the images that have "colours/" in their path
df_new[df_new['image'].str.contains("colours/")]['mse'].mean()
# finding the mean mse of the images that have "colours_blurred/" in their path
df_new[df_new['image'].str.contains("colours_blurred/")]['mse'].mean()
# finding the mean mse of the images that have "colours_noisy/" in their path
df_new[df_new['image'].str.contains("colours_noisy/")]['mse'].mean()
# finding the mean mse of the images that have "colours_noisy_gaussian/" in their path
df_new[df_new['image'].str.contains("colours_noisy_gaussian/")]['mse'].mean()